In [1]:
import numpy as np
import matplotlib.pyplot as plt

%tensorflow_version 1.x
import tensorflow as tf
print("Using tensorflow v" + str(tf.__version__))

import sys
!{sys.executable} -m pip install -U deepposekit pyrealsense2

# HOW TO USE THIS SCRIPT:

# 1. Update the source
# 2. Make sure annotator is uncommented
# 3. Maybe change the text scaling

from deepposekit import Annotator
from deepposekit.io import VideoReader, DataGenerator, initialize_dataset, TrainingGenerator, BaseGenerator
from deepposekit.io import ImageGenerator, VideoWriter
from deepposekit.io.utils import merge_new_images
from deepposekit.annotate import KMeansSampler
from deepposekit.augment import FlipAxis
from deepposekit.models import StackedDenseNet, DeepLabCut,StackedHourglass,LEAP
from deepposekit.models import load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from deepposekit.callbacks import Logger, ModelCheckpoint
from scipy.signal import find_peaks
#import pyrealsense2 as rs

import tqdm
import cv2

import imgaug.augmenters as iaa
import imgaug as ia


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

#source = 'chick-toy'
#HOME = f"{source}/"

from os.path import expanduser
try:
    import google.colab
    IN_COLAB = True
    data_path = '/content/drive/MyDrive/GitHub IU/bam/deepposekit_clone/deepposekit_data_custom/'
    source = 'human'
    HOME = data_path + f"{source}/"
    print(HOME, source)
        
except:
    IN_COLAB = False
    data_path = '/content/drive/MyDrive/GitHub IU/bam/deepposekit_clone/deepposekit_data_custom/'
    source = 'human'
    HOME = data_path + f"{source}/"
    print(HOME, source)

TensorFlow 1.x selected.
Using tensorflow v1.15.2
     |████████████████████████████████| 71kB 4.5MB/s 
     |████████████████████████████████| 70.8MB 40kB/s 
  Created wheel for deepposekit: filename=deepposekit-0.3.9-cp36-none-any.whl size=105114 sha256=d10fc872fc275802387dafc9fe6a52cc09dd3c806a10873d08f397f58ed90dcb
  Stored in directory: /root/.cache/pip/wheels/63/92/3e/96682d235db0100cb3f86a8ddd677756a22fd04f0ee1fe3936
Successfully built deepposekit
Num GPUs Available:  1
/content/drive/MyDrive/GitHub IU/bam/deepposekit_clone/deepposekit_data_custom/human/ human


In [2]:
HOME + f'{source}_annotation_set.h5'

'/content/drive/MyDrive/GitHub IU/bam/deepposekit_clone/deepposekit_data_custom/human/human_annotation_set.h5'

In [ ]:
cap = cv2.VideoCapture(HOME + f'{source}_raw.mp4')
 
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter(HOME + video_file_path + 'pose_human_test_resized.mp4',fourcc, 30, resize_shape)

out = VideoWriter(HOME + f'{source}.mp4', resize_shape, 'mp4v', 30.0, color=True)

while True:
    ret, frame = cap.read()
    if ret == True:
        b = cv2.resize(frame,resize_shape,fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
        out.write(b)
    else:
        break
    
cap.release()
out.close()
cv2.destroyAllWindows()

In [7]:
def annotate_dataset(overwrite=False):
    print("[INFO] Preparing Data")
    # batch_size must be large else kmeans can't be performed
    reader = VideoReader(HOME + f'{source}.mp4', batch_size=100, gray=True)

    randomly_sampled_frames = []
    for idx in tqdm.tqdm(range(len(reader)-1)):
        batch = reader[idx]
        random_sample = batch[np.random.choice(batch.shape[0], 10, replace=False)]
        randomly_sampled_frames.append(random_sample)
    reader.close()

    randomly_sampled_frames = np.concatenate(randomly_sampled_frames)
    kmeans = KMeansSampler(n_clusters=10, max_iter=100, n_init=20, batch_size=100, verbose=True)
    kmeans.fit(randomly_sampled_frames)
    kmeans_sampled_frames, kmeans_cluster_labels = kmeans.sample_data(randomly_sampled_frames, n_samples_per_label=50)

    try:
        initialize_dataset(
            images=kmeans_sampled_frames,
            datapath=HOME + f'{source}_annotation_set.h5',
            skeleton=HOME + 'skeleton.csv',
            overwrite=overwrite
        )
    except OSError:
        print("[INFO] Dataset Exists - Passing.")

    # THIS CANNOT BE DONE FROM WITHIN GOOGLE COLAB. USE PYCHARM or an IDE
    Annotator(datapath = HOME + f'{source}_annotation_set.h5',
                   dataset ='images',
                   skeleton = HOME + 'skeleton.csv',
                   shuffle_colors = False,
                   text_scale = 0.2).run()



In [8]:

#annotate_dataset(overwrite=True)

In [9]:
import os
os.listdir(HOME)

['skeleton.csv',
 'pose_human_test.mp4',
 'best_model_densenet.h5',
 'predictions.npy',
 'human.mp4',
 'human_annotation_set.h5']

In [10]:
HOME + f'{source}_annotation_set.h5'

'/content/drive/MyDrive/GitHub IU/bam/deepposekit_clone/deepposekit_data_custom/human/human_annotation_set.h5'

In [11]:

def prepare_model():

    print("Loading Data Generator")
    data_generator = DataGenerator(HOME + f'{source}_annotation_set.h5', mode="annotated")

    print("Creating Data Augmenter")
    augmenter = []
    augmenter.append(FlipAxis(data_generator, axis=0))  # flip image up-down
    augmenter.append(FlipAxis(data_generator, axis=1))  # flip image left-right

    sometimes = []
    sometimes.append(iaa.Affine(scale={"x": (0.95, 1.05), "y": (0.95, 1.05)},
                                translate_percent={'x': (-0.05, 0.05), 'y': (-0.05, 0.05)},
                                shear=(-8, 8),
                                order=ia.ALL,
                                cval=ia.ALL,
                                mode=ia.ALL)
                     )
    sometimes.append(iaa.Affine(scale=(0.8, 1.2),
                                mode=ia.ALL,
                                order=ia.ALL,
                                cval=ia.ALL)
                     )
    augmenter.append(iaa.Sometimes(0.75, sometimes))
    augmenter.append(iaa.Affine(rotate=(-180, 180),
                                mode=ia.ALL,
                                order=ia.ALL,
                                cval=ia.ALL)
                     )
    augmenter = iaa.Sequential(augmenter)

    print("Creating Training Generator")
    train_generator = TrainingGenerator(generator=data_generator,
                                        downsample_factor=3,
                                        augmenter=augmenter,
                                        sigma=3,
                                        validation_split=0.3,
                                        use_graph=True,
                                        random_seed=1,
                                        graph_scale=1)
    print(train_generator.get_config())
    train_generator.on_epoch_end()

    with tf.device("gpu:0"):
        print("[INFO] Preparing Model")
        # SELECT MODEL

        # model = StackedDenseNet(train_generator, n_stacks=5, growth_rate=32, pretrained=True)
        # model = DeepLabCut(train_generator, backbone="resnet50")
        # model = DeepLabCut(train_generator, backbone="mobilenetv2", alpha=0.75) # Increase alpha to improve accuracy
        model = DeepLabCut(train_generator, backbone="densenet121")
        # model = LEAP(train_generator)
        # model = StackedHourglass(train_generator)

        model.get_config()
        reduce_lr = ReduceLROnPlateau(monitor="loss", factor=0.2, verbose=1, patience=20)

        model_checkpoint = ModelCheckpoint(
            HOME + "best_model_densenet.h5",
            monitor="loss",
            # monitor="loss" # use if validation_split=0
            verbose=1,
            save_best_only=True,
        )
        early_stop = EarlyStopping(
            monitor="loss",
            # monitor="loss" # use if validation_split=0
            min_delta=0.001,
            patience=100,
            verbose=1
    )

        print("Training model...")
        callbacks = [early_stop, reduce_lr, model_checkpoint]
        model.fit(
            batch_size=1,
            validation_batch_size=5,
            callbacks=callbacks,
            epochs=1,
            steps_per_epoch=None,
        )

        model = load_model(
            HOME + "best_model_densenet.h5",
            augmenter=augmenter,
            generator=data_generator,
        )

        model.fit(
            batch_size=1,
            validation_batch_size=5,
            callbacks=callbacks,
            epochs=1,
            steps_per_epoch=None,
        )

In [12]:

prepare_model()


Loading Data Generator


/usr/local/lib/python3.6/dist-packages/deepposekit/io/DataGenerator.py:81: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.annotated = np.all(h5file["annotated"].value, axis=1)


Creating Data Augmenter
Creating Training Generator
[INFO] Preparing Model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
29089792/29084464 [==============================] - 0s 0us/step
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Training model...


/usr/local/lib/python3.6/dist-packages/deepposekit/models/engine.py:145: UserWarning: 
Automatically compiling with default settings: model.compile('adam', 'mse')
Call model.compile() manually to use non-default settings.

  """\nAutomatically compiling with default settings: model.compile('adam', 'mse')\n"""
/usr/local/lib/python3.6/dist-packages/deepposekit/models/engine.py:161: UserWarning: No validation set detected, so validation step will not be run and `val_loss` will not be available.
  "No validation set detected, so validation step will not be run and `val_loss` will not be available."


8/9 [=========================>....] - ETA: 5s - loss: 11.3334 
Epoch 00001: loss improved from inf to 11.15484, saving model to /content/drive/MyDrive/GitHub IU/bam/deepposekit_clone/deepposekit_data_custom/human/best_model_densenet.h5
9/9 [==============================] - 54s 6s/step - loss: 11.1548
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
8/9 [=========================>....] - ETA: 2s - loss: 9.8932
Epoch 00001: loss improved from 11.15484 to 9.92928, saving model to /content/drive/MyDrive/GitHub IU/bam/deepposekit_clone/deepposekit_data_custom/human/best_model_densenet.h5
9/9 [==============================] - 34s 4s/step - loss: 9.9293


In [16]:

def create_video():
    print("[INFO] Creating Output Video")

    with tf.device("gpu:0"):
        model = load_model(HOME + 'best_model_densenet.h5')

    model_size = tuple(model.input_shape[:2])
    print(model_size, model_size[::-1])
    model_size = model_size[::-1] 

    print("Reading Video...")
    reader = VideoReader(HOME + f'{source}.mp4', batch_size=1, gray=True)
    predictions = model.predict(reader, verbose=1)
    np.save(HOME + 'predictions.npy', predictions)
    #############################################

    data_generator = DataGenerator(HOME + f'{source}_annotation_set.h5')
    predictions = predictions[..., :2]
    print(predictions.shape)

    cmap = plt.cm.hsv(np.linspace(0, 1, data_generator.keypoints_shape[0]))[:, :3][:, ::-1] * 255

    writer = VideoWriter(HOME + f'{source}_predicted.mp4', model_size, 'mp4v', 30.0, color=True)
    for frame, keypoints in tqdm.tqdm(zip(reader, predictions)):
        frame = frame[0]
        frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
        for i, node in enumerate(data_generator.graph):
            if node >= 0:
                pt1 = keypoints[i]
                pt2 = keypoints[node]
                cv2.line(frame, (pt1[0], pt1[1]), (pt2[0], pt2[1]), (0, 0, 255), 1, cv2.LINE_AA)
        for i, keypoint in enumerate(keypoints):
            keypoint = keypoint.astype(int)
            cv2.circle(frame, (keypoint[0], keypoint[1]), 1, tuple(cmap[i]), -1, cv2.LINE_AA)
        writer.write(frame)

    writer.close()
    reader.close()


In [17]:
create_video()
print("[INFO] Process Finished")

[INFO] Creating Output Video
(320, 640) (640, 320)
Reading Video...
275/275 [==============================] - 21s 75ms/step
(275, 7, 2)


/usr/local/lib/python3.6/dist-packages/deepposekit/io/DataGenerator.py:81: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.annotated = np.all(h5file["annotated"].value, axis=1)
275it [00:00, 408.22it/s]

[INFO] Process Finished
